In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()

In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d cdart99/food20dataset

 99% 431M/435M [00:04<00:00, 114MB/s] 
100% 435M/435M [00:04<00:00, 110MB/s]


In [5]:
! ls
! mkdir dataset
! unzip food20dataset.zip -d dataset

adc.json  food20dataset.zip  kaggle.json  sample_data
Archive:  food20dataset.zip
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (1).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (10).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (11).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (12).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (13).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (14).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (15).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (16).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (17).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (18).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (19).jpg  
  inflating: dataset/food20dataset/test_set/biriyani/biriyanitest (2).jp

In [6]:
import seaborn as sns
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix


In [15]:
#from scipy.misc import imread
categories = os.listdir('/content/dataset/food20dataset/test_set')

img_size= 224

def data_gen(data_dir):
    data=[]
    for category in categories:
        path= os.path.join(data_dir,category)
        class_num= categories.index(category)
        #print(path)
        for img in os.listdir(path):
            try:
                img_arr= cv2.imread(os.path.join(path,img))
                plt.imshow(img_arr)
                resized_arr= cv2.resize(img_arr,(img_size,img_size))
                #img_arr= cv2.imread(os.path.join(path,img))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
        
    return np.array(data)

In [ ]:
train_data= data_gen('/content/dataset/food20dataset/train_set')
#val_data= data_gen('/content/dataset/food20dataset/test_set')
categories

['paniyaram',
 'dosa',
 'gulab jamun',
 'chaat',
 'dhokla',
 'kathi roll',
 'tandoori chicken',
 'chappati',
 'idly',
 'meduvadai',
 'ven pongal',
 'biriyani',
 'noodles',
 'vada pav',
 'poori',
 'bisibelebath',
 'samosa',
 'halwa',
 'butternaan',
 'upma']

In [17]:
lenofimage = len(train_data)

In [18]:
X=[]
y=[]

for categories, label in train_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [19]:
X.shape

(1400, 150528)

In [20]:
X = X/255.0

In [21]:
y=np.array(y)

In [22]:
for i in range(len(y)):
  if(y[i]!= 1):
    y[i] = 0
    
from sklearn.model_selection import train_test_split
xx, aa, yy, bb = train_test_split(X,y)

In [23]:
temp1=list(zip(xx,yy))
temp2=list(zip(aa,bb))

train = {}
for elem in temp1:
    try:
        train[elem[1]].append(elem[0])
    except KeyError:
        train[elem[1]] = [elem[0]]

test = {}
for elem in temp2:
    try:
        test[elem[1]].append(elem[0])
    except KeyError:
        test[elem[1]] = [elem[0]]


In [26]:
from sklearn import svm
model = svm.SVC(kernel='linear', degree=3, C=1, decision_function_shape='ovo').fit(xx,yy)


In [27]:
pred = model.predict(aa)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy on unknown data is",accuracy_score(pred,bb))

Accuracy on unknown data is 0.9257142857142857


In [ ]:
!pip install qiskit

import qiskit
from qiskit import transpile, assemble
from qiskit.visualization import *

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import Aer
from qiskit.ml.datasets import ad_hoc_data, sample_ad_hoc_data, breast_cancer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.aqua.utils import split_dataset_to_data_and_labels, map_label_to_class_name
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/ml/__init__.py:40: DeprecationWarning: The package qiskit.ml is deprecated. It was moved/refactored to qiskit_machine_learning (pip install qiskit-machine-learning). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('ml', 'qiskit_machine_learning', 'qiskit-machine-learning')


In [ ]:
'''feature_dim = 2 
seed = 10598  # Setting seed to ensure reproducable results

feature_map_bc = ZZFeatureMap(feature_dimension=feature_dim, reps=2, entanglement='linear')
qsvm_bc = QSVM(feature_map_bc, train, test)

backend = Aer.get_backend('qasm_simulator')
quantum_instance_bc = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed)
result_bc = qsvm_bc.run(quantum_instance_bc)
print(f"Testing Accuracy: {result_bc['testing_accuracy'] * 100}%")'''

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/quantum_instance.py:137: DeprecationWarning: The class qiskit.aqua.QuantumInstance is deprecated. It was moved/refactored to qiskit.utils.QuantumInstance (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit-terra')
